# Neuromodulation example

This neuromodulation creates a small network of neurons. We also use the reaction diffusion model by Anu G Nair 2015.

To generate the ```reaction_diffusion.json``` file in ```data/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026/``` from the xml file we run ```data/convert_sbml_to_json.sh```

To get the RxD functionality of the ```DA_syn``` we must specify the "RxD" block in the connectivity block of the network configuration. See ```data/connectivity.json```

```
    "channel_parameters":
		"RxD": {
		    "species_name": "DA",
		    "flux_variable": "open",
		    "region": "internal",
            "weight_scaling": 1e9,
		},

     ...
     }
```

## Network setup

We have two neurons. The first neuron (Neuron 0) receives external input (cortical from t=0s and DA from t=0.1s). The cortical input will activate the first neuron, and through activation of synapses on the second neuron, we will see the dopamine level increase in the second neuron (Neuron 1).

The first neuron also receives direct DA activation from external input (starting at 100ms).

In [ ]:
import os
from snudda import Snudda

neuron_path = os.path.join("data", "dspn")
network_path = os.path.join("networks", "neuromodulation_example_anu_with_real_dspn")
connectivity_path = os.path.join("data", "connectivity.json")

snudda = Snudda(network_path=network_path)
snudda.init_tiny(neuron_paths=neuron_path, neuron_names="neuron", number_of_neurons=[2], 
                 connection_config=connectivity_path, random_seed=12345)
snudda.create_network()

In [ ]:
input_config = os.path.join("data", "input_v2.json")
snudda.setup_input(input_config=input_config)

In [ ]:
mech_dir = os.path.join("..", "..", "..", "..", "BasalGangliaData", "data", "neurons", "mechanisms")
sim = snudda.simulate(time=0, mech_dir=mech_dir, verbose=True, sample_dt=0.00005)

### Add PKA and DA recordings
Here we add recordings from a compartment that receives synaptic input from its neighbour. This has been checked by looking at the synapse_connection matrix (snudda_load command).

In [ ]:
sim.add_rxd_concentration_recording(species="PKA", neuron_id=1,
                                    region="dend_internal",
                                    sec_type="dend",
                                    sec_id=4,
                                    sec_x=0.25)

In [ ]:
sim.add_rxd_concentration_recording(species="DA", neuron_id=1,
                                    region="dend_internal",
                                    sec_type="dend",
                                    sec_id=4,
                                    sec_x=0.25)

### Add additional PKA and DA recorings, for input

In [ ]:
sim.add_rxd_concentration_recording(species="DA", neuron_id=0,
                                    region="dend_internal",
                                    sec_type="dend",
                                    sec_id=7,
                                    sec_x=0.233)

In [ ]:
sim.add_rxd_internal_concentration_recording_all(species="PKA", neuron_id=0)

In [ ]:
sim.run(t=200)

In [ ]:
sim.record.write()

## Load the data and plot

In [ ]:
from snudda.utils import SnuddaLoadNetworkSimulation

output_file = os.path.join(network_path, "simulation", "output.hdf5")
nd = SnuddaLoadNetworkSimulation(output_file)
time = nd.get_time()
data_pka = nd.get_data("PKA", 1)[0][1]
data_da = nd.get_data("DA", 1)[0][1]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKA", 0)[0][0]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_da, label="DA")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_pka, label="PKA")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_da_external, label="DA from external?")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
from snudda.plotting import SnuddaPlotSpikeRaster2
fig_file_raster = f"spike-raster.png"

time_range_zoom = (0,0.1)
spr = SnuddaPlotSpikeRaster2(network_path=network_path)

spr.plot_spike_raster(fig_file=fig_file_raster, time_range=time_range_zoom)

In [ ]:
%matplotlib inline

sim_file = os.path.join(network_path, "simulation", "output.hdf5")

from snudda.plotting.plot_traces import PlotTraces
pt = PlotTraces(output_file=sim_file)
# Use trace_id to specify which traces
ax = pt.plot_traces(offset=0, time_range=(0,0.5),fig_size=(10,4))

## TODO!! Also add ability to have external synapses affect RxD concentrations

In [ ]:
sim.neurons[0].icell.soma[0](0.5).naf_ms.gbar

In [ ]:
sim.neurons[0].icell.soma[0](0.5).pas.g